# Analyzing Sepher

The sepher dataset is expansive and has a lot of features that we will be able to use to make our predictions.
We potentially have the problem of scoping in resolved to a decent degree too.
The dataset consist of a lot of data that spans across states.
If we pick a hurricane coordinate and manage to figure out which state and countries it affects, we may be able to leverage this to output data about how many households could potentially be affected by this.

The `uszips` dataset will allow us to do this, possible because the state and county is already present in the dataset. It can be attributed to the following resource:
- <cite>https://simplemaps.com/data/us-zips</cite>

The `sepher` dataset is attributed to the following resource:
- <cite>
Tedesco, M., C. Hultquist, S. E. Char, C. Constantinides, T. Galjanic, A. D. Sinha. 2021. Socio-Economic, Physical, Housing, Eviction, and Risk dataset, version 2 (SEPHER 2.0), Preliminary Release. https://doi.org/10.7927/r6yw-xw73. Accessed 09/12/2024.
</cite>

In [2]:
import duckdb

In [4]:
duckdb.execute("CREATE TABLE sepher AS SELECT * FROM '../data/sepher2.0_cleaned.csv'")

In [ ]:
duckdb.execute("DESCRIBE sepher").df()

In [67]:
duckdb.execute("""SELECT COUNT(1) FROM sepher WHERE STATEABBRV = 'FL' AND COUNTY = 'MIAMI-DADE' LIMIT 10""").df()

,count(1)
0,518


The percentage of people in housing structures that have more than 10 units or live in mobile homes may be more in danger of being affected by the hurricane. This information may prove to be useful during the analysis as these people may be more affected that the rest.

The `E_HU_*` statistic gives us an estimate of the housing units in this particular section of Miami-Dade. We can leverage this to compute our estimate number of houses that fall into the `EP_*` statistics and provide a distribution of the people that are at risk of being affected by the hurricane.

In [84]:
duckdb.execute("""
SELECT
    EP_MUNIT_2000,
    EP_MUNIT_2010,
    EP_MUNIT_2014,
    EP_MUNIT_2016,
    EP_MUNIT_2018,
    EP_MOBILE_2000,
    EP_MOBILE_2010,
    EP_MOBILE_2014,
    EP_MOBILE_2016,
    EP_MOBILE_2018,
    E_HU_2000,
    E_HU_2010,
    E_HU_2014,
    E_HU_2016,
    E_HU_2018
FROM
    sepher
WHERE
    STATEABBRV = 'FL' AND
    COUNTY = 'MIAMI-DADE'
LIMIT 10
""").df()

,EP_MUNIT_2000,EP_MUNIT_2010,EP_MUNIT_2014,EP_MUNIT_2016,EP_MUNIT_2018,EP_MOBILE_2000,EP_MOBILE_2010,EP_MOBILE_2014,EP_MOBILE_2016,EP_MOBILE_2018,E_HU_2000,E_HU_2010,E_HU_2014,E_HU_2016,E_HU_2018
0,NA,79.7261,82.5,84.8,87.4,NA,0,0,0,0,NA,1899,2198,2276,2451
1,21.44397,63.596,61.9,59.6,62,32.00431,17.0443,11,15.2,16,928,2030,2277,2154,2081
2,96.81118,95.3667,95.2,96.4,96.6,0,0,0,0,0,2791,4921,6340,6528,6239
3,88.95277,80.48,79.4,80.5,80.7,0,0,0,0,0,8871,2833,2786,2599,2584
4,NA,100,97.9,97.6,98,NA,0,0,0,0,NA,931,1620,1910,1906
5,NA,88.568,93.5,86.4,82.2,NA,0,0,0.3,0.3,NA,3359,3767,3762,3669
6,NA,54.2844,53.6,54.1,57,NA,0,0.4,0.3,0,NA,2264,2308,2306,2193
7,NA,93.8869,92.9,94.4,93.1,NA,0,0,0,0,NA,1963,2196,1847,1818
8,NA,95.5617,95.7,95.5,95.1,NA,0.6207,0,0,0,NA,3222,3524,3716,3563
9,NA,89.4285,92,89.4,91,NA,1.1068,1.6,0,0,NA,4427,4855,4514,4392


Finding out statistics on coastal flooding for these particular houses.

In [74]:
duckdb.execute("""
SELECT
    CFLD_AFREQ,
    CFLD_EXPB,
    CFLD_EXPP,
    CFLD_EXPPE,
    CFLD_EXPT,
    CFLD_HLRB,
    CFLD_HLRP,
    CFLD_HLRR,
    CFLD_EALB,
    CFLD_EALP,
    CFLD_EALPE,
    CFLD_EALT,
    CFLD_EALS,
    CFLD_EALR,
    CFLD_RISKS,
    CFLD_RISKR
FROM
    sepher
WHERE
    STATEABBRV = 'FL' AND
    COUNTY = 'MIAMI-DADE'
LIMIT 10
""").df()

,CFLD_AFREQ,CFLD_EXPB,CFLD_EXPP,CFLD_EXPPE,CFLD_EXPT,CFLD_HLRB,CFLD_HLRP,CFLD_HLRR,CFLD_EALB,CFLD_EALP,CFLD_EALPE,CFLD_EALT,CFLD_EALS,CFLD_EALR,CFLD_RISKS,CFLD_RISKR
0,2.797475,329667000,2347,17837200000,18166867000,8.20385e-05,4.843119e-06,VERY LOW,75658.79,0.03179834,241667.4,317326.2,22.48235,RELATIVELY HIGH,21.47661,RELATIVELY HIGH
1,2.809475,428427633,3580.329,27210500198,27638927831,4.470987e-05,3.059377e-06,VERY LOW,53815.33,0.0307738,233880.8,287696.2,21.75961,RELATIVELY HIGH,22.32064,RELATIVELY HIGH
2,2.797475,927069146,6747.272,51279268565,52206337711,1.081672e-05,7.637938e-07,VERY LOW,28052.66,0.01441685,109568.1,137620.8,17.01771,RELATIVELY MODERATE,16.03019,RELATIVELY MODERATE
3,2.807475,422311712,2670.033,20292247590,20714559302,4.192347e-05,2.518295e-06,VERY LOW,49705.7,0.01887726,143467.2,193172.9,19.05411,RELATIVELY MODERATE,13.89451,RELATIVELY MODERATE
4,2.707745,174749367,838.6436,6373691418,6548440785,4.728984e-06,4.954708e-07,VERY LOW,2237.646,0.001125132,8551,10788.65,7.28331,RELATIVELY LOW,6.227433,RELATIVELY LOW
5,2.797475,446384955,3831.957,29122873271,29569258226,3.963781e-06,2.278283e-07,VERY LOW,4949.773,0.002442275,18561.29,23511.06,9.442723,RELATIVELY LOW,9.498337,RELATIVELY LOW
6,2.807475,621055786,4215.002,32034013012,32655068799,4.370024e-05,3.061227e-06,VERY LOW,76195.66,0.03622506,275310.5,351506.1,23.2622,RELATIVELY HIGH,19.78787,RELATIVELY HIGH
7,2.797475,282085434,1460.891,11102774134,11384859567,4.027927e-05,2.093064e-06,VERY LOW,31785.45,0.008553949,65010.01,96795.47,15.13416,RELATIVELY MODERATE,16.93712,RELATIVELY MODERATE
8,2.797475,380092085,3309.965,25155734498,25535826582,2.8261e-05,1.662722e-06,VERY LOW,30049.86,0.01539605,117010,147059.8,17.39821,RELATIVELY MODERATE,17.91048,RELATIVELY MODERATE
9,2.807475,660722000,3761,28583600000,29244322000,4.611715e-05,4.140853e-06,VERY LOW,85545.48,0.0437229,332294.1,417839.5,24.64202,RELATIVELY HIGH,24.24709,RELATIVELY HIGH


The following attributes may prove to be useful in our research too as they address other potential households that are at risk in the case of a hurricane.
- `EP_CROWD_`: Occupied housing with more people than rooms.
- `EP_NOVEH_`: Households with no vehicle available estimate.
- `EP_GROUPQ_`: Percentage of people in group quarters estimate.

In [86]:
duckdb.execute("""
SELECT
    EP_CROWD_2000,
    EP_CROWD_2010,
    EP_CROWD_2014,
    EP_CROWD_2016,
    EP_CROWD_2018,
    EP_NOVEH_2000,
    EP_NOVEH_2010,
    EP_NOVEH_2014,
    EP_NOVEH_2016,
    EP_NOVEH_2018,
    EP_GROUPQ_2000,
    EP_GROUPQ_2010,
    EP_GROUPQ_2014,
    EP_GROUPQ_2016,
    EP_GROUPQ_2018
FROM
    sepher
WHERE
    STATEABBRV = 'FL' AND
    COUNTY = 'MIAMI-DADE'
LIMIT 10
""").df()

,EP_CROWD_2000,EP_CROWD_2010,EP_CROWD_2014,EP_CROWD_2016,EP_CROWD_2018,EP_NOVEH_2000,EP_NOVEH_2010,EP_NOVEH_2014,EP_NOVEH_2016,EP_NOVEH_2018,EP_GROUPQ_2000,EP_GROUPQ_2010,EP_GROUPQ_2014,EP_GROUPQ_2016,EP_GROUPQ_2018
0,NA,2.0702,0,2.2,3.3,NA,9.1809,11.9,6.6,5.1,NA,0,0,0,0
1,22.96,6.7567,6.7,7.6,11.4,13.98,5.5405,17.1,13.7,14.1,16.10127,4.3073,5,2.2,0
2,3.4,1.9851,2.4,1.3,2.2,7.25,4.9273,5.2,5.8,5.9,0,0,0,0,0
3,11.74,0.1576,4,7.2,12.6,17.73,1.4972,2.3,1.9,1.6,0.263504,0,0.2,0,0
4,NA,0,0,2.2,2.9,NA,0,2.2,3.4,7.6,NA,0,0.3,0,0
5,NA,1.2809,2,1.9,1,NA,15.1579,14.6,11.8,10.4,NA,0,0,0,0
6,NA,0.9036,0.8,0.5,2.4,NA,4.4578,3,1.6,3.3,NA,0,0,0,0
7,NA,1.8641,10.5,7.5,6.4,NA,1.8641,6.5,8.4,7.5,NA,0,0,0,0
8,NA,4.3107,0,1,2.1,NA,10.2756,8.6,7.8,10.4,NA,0,0,0,0
9,NA,0,5.4,5.7,5.1,NA,15.4293,11.8,5.8,5.7,NA,0,0,0,0


As one can see, some of the statistics we're receiving from the Coastal Flooding (CFLD) statistics seem to be providing us a relatively high hazard type risk index rating. This is an important statistic to follow as it can be used to understand what factors make it relatively high in terms of risk. These also seem to have been retrieved from the FEMA NRI dataset.

In [83]:
duckdb.execute("""
SELECT
    HRCN_EVNTS,
    HRCN_AFREQ,
    HRCN_EXPB,
    HRCN_EXPP,
    HRCN_EXPPE,
    HRCN_EXPA,
    HRCN_EXPT,
    HRCN_HLRB,
    HRCN_HLRP,
    HRCN_HLRA,
    HRCN_HLRR,
    HRCN_EALB,
    HRCN_EALP,
    HRCN_EALPE,
    HRCN_EALA,
    HRCN_EALT,
    HRCN_EALS,
    HRCN_EALR,
    HRCN_RISKS,
    HRCN_RISKR
FROM
    sepher
WHERE
    STATEABBRV = 'FL' AND
    COUNTY = 'MIAMI-DADE' AND
    HRCN_HLRR = 'RELATIVELY HIGH'
""").df()

,HRCN_EVNTS,HRCN_AFREQ,HRCN_EXPB,HRCN_EXPP,HRCN_EXPPE,HRCN_EXPA,HRCN_EXPT,HRCN_HLRB,HRCN_HLRP,HRCN_HLRA,HRCN_HLRR,HRCN_EALB,HRCN_EALP,HRCN_EALPE,HRCN_EALA,HRCN_EALT,HRCN_EALS,HRCN_EALR,HRCN_RISKS,HRCN_RISKR
0,29,0.2899957,1607000,0,0,1263030,2870030,0.0001180271,5.367216e-07,0.06593869,RELATIVELY HIGH,53.76783,0,0,23623.64,23677.41,11.42128,RELATIVELY LOW,NA,INSUFFICIENT DATA


We also seem to have statistics on particular locations that have higher risk of the hurricane hazard (HRCN). However, the HRCN_HLRB statistic which stands for "Hurricane - Historic Loss Ratio - Buildings" doesn't seem to indicate that the hurricane will end up damaging the houses.

Other statistics that may be of interest in this study are:
- `RFLD_*` which are useful to understand the impact of Riverine Flooding.
- `SWND_*` which is an indicator for strong wind
- `TRND_*` which is an indicator for Tornados.
- `TSUN_*` which is an indicator for Tsunamis.